In [1]:
import numpy as np
import pandas as pd
import sys
sys.path
sys.path.append('./scripts/')
import POMDPLearn as pom

# Import dataset

- split into train test dataset

In [2]:
df_POMDP = pd.read_csv('./Datasets/POMDP_train_test.csv')

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
X = df_POMDP.values
y = df_POMDP.medication
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.33, random_state=0)
sss.get_n_splits(X, y)

print(sss)       

for train_index, test_index in sss.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    dfTrain, dfTest = df_POMDP.iloc[train_index].copy(), df_POMDP.iloc[test_index].copy()

StratifiedShuffleSplit(n_splits=1, random_state=0, test_size=0.33,
            train_size=None)


In [5]:
dfTest.head(3)

,medication,state0,state1,state2,state3,state4,state5,state6,state7,state8,...,obs_lat_14,obs_lat_15,obs_lat_16,obs_lat_17,obs_lat_18,obs_lat_19,age,Gender,Parkinsons,Tremors
257,1,0,0,0,0,0,0,0,0,0,...,6,0,6,0,6,0,48,0,1,1
217,1,0,0,0,0,0,0,0,0,0,...,6,4,6,4,5,0,48,0,1,0
64,1,0,0,0,0,0,0,0,0,0,...,4,3,6,0,0,2,48,0,0,0


In [6]:
pomdp_train_dataset = pom.POMDPDataset(dfTrain)
pomdp_test_dataset = pom.POMDPDataset(dfTest)

196it [00:00, 51528.37it/s]
196it [00:00, 128050.40it/s]
98it [00:00, 68166.13it/s]
98it [00:00, 93418.59it/s]


## POMDP dataset

- training a pomdp model using the training dataset 
- testing a pomdp model using the testing dataset

In [8]:
pomdpPark = pom.POMDP(states=pomdp_train_dataset.unique_states,
                      actions=pomdp_train_dataset.unique_actions,
                      observations=pomdp_train_dataset.unique_observations,
                      rewards=None,T=None, discount=0.9,
                      horizon=pomdp_train_dataset.horizon,
                      action_invariant=True)

In [9]:
pomdpPark.trainPOMDP(POMDPDataset=pomdp_train_dataset)

Learning the transition and observation matrix ...
EM iteration 1, loglik = -13279.5967
EM iteration 2, loglik = -7191.6483
EM iteration 3, loglik = -7191.6483
Learning the state rewards ...
Learning the transition matrix of the action MDP ... 

EM iteration 1, loglik = -2379.8369
EM iteration 2, loglik = -556.4701
EM iteration 3, loglik = -556.4701
Learning the action rewards ... 

Using the multiplicative model to learn state action pair rewards ...


### Solving the POMDP model to obtain alpha vectors

In [10]:
pomdpPark.POMDPSolve()

POMDP solved!


In [11]:
#the alpha vectors
actions = ['No medication','Medication']
for i in range(len(pomdpPark.alpha_vectors)):
    print('Action of '+actions[i] +': ', 
          pomdpPark.alpha_vectors[i].action)
    print('Vector: ', pomdpPark.alpha_vectors[i].v)

Action of No medication:  0
Vector:  [10.69512288  8.686135    8.98938883]
Action of Medication:  1
Vector:  [ 8.69512288  7.3786216  10.98938883]


## Initial Belief model - Naive Bayes

In [12]:
from sklearn.naive_bayes import GaussianNB

cols = ['age','Gender', 'Parkinsons', 'Tremors']

X_train = dfTrain[cols].values
y_train = dfTrain['medication'].values

#initial belief model
gnb = GaussianNB()
clf = gnb.fit(X_train, y_train)

pomdp_train_dataset.initial_belief_model = clf
pomdp_train_dataset.baseline_features = cols

#probability of medication
y_prob = clf.predict_proba(X_train)[:,1]

#initial beliefs of each individual
pomdp_train_dataset.initial_beliefs = np.vstack((np.vstack((1-y_prob,np.zeros(len(dfTrain)))),y_prob)).T

### Getting POMDP recommended actions and updated beliefs over horizon

In [13]:
rec_actions,updated_beliefs = pomdpPark.getRecActions(POMDPDataset=pomdp_train_dataset)

In [14]:
## accuracy of finding who was using medication or not in training set

from sklearn.metrics import accuracy_score

y_true = dfTrain.medication.values
y_pred = (np.sum(rec_actions,axis=1)>0)*1

accuracy_score(y_true, y_pred)

0.8673469387755102

## Test set

In [15]:
X_test = dfTest[pomdp_train_dataset.baseline_features].values
y_test = dfTest['medication'].values

#probability of medication
y_prob = pomdp_train_dataset.initial_belief_model.predict_proba(X_test)[:,1]

#test set initial beliefs
pomdp_test_dataset.initial_beliefs = np.vstack((np.vstack((1-y_prob,np.zeros(len(dfTest)))),y_prob)).T

### Getting POMDP recommended actions and updated beliefs over horizon

In [16]:
rec_actions,updated_beliefs = pomdpPark.getRecActions(POMDPDataset=pomdp_test_dataset)

In [17]:
## accuracy of finding who was using medication or not in training set

from sklearn.metrics import accuracy_score

y_true = dfTest.medication.values
y_pred = (np.sum(rec_actions,axis=1)>0)*1

accuracy_score(y_true, y_pred)

0.8469387755102041

# Analysis of results

- Analyzing the true actions vs the recommended actions

In [18]:
#true actions
pomdp_test_dataset.actions_data[:10,:]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [20]:
rec_actions[:10,:20] #PODMPD can suggest an additional action because of last updated belief

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In the above 10 cases the, all 10 participants were under medication
2 of them at the 15th epoch of the horizon and 8 at the 10th.

The POMDP model correctly identified 8 out of 10 as needing medication.

Interstingly from the ones that are correctly identified the medication is recommended earlier rather than later, meaning that the individuals would benefit from the medication earlier.